In [26]:
%pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 18.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import shutil

import os
import pandas as pd
from scipy.ndimage import zoom
import SimpleITK as sitk

#import nibabel as nib
import glob

In [17]:
#shutil.rmtree("/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_2/images")

In [5]:
!cp /mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/* /mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_3/images

In [5]:
!cp /mnt/ai_neuro/august_kun/data/UnlabeledCase/* /mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_1/images

In [12]:
ex_i=sorted(glob.glob("/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_3/images/*.nii.gz"))[:100]
ex_m=sorted(glob.glob("/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_1/contours/*"))[:100]

In [3]:
len(ex_i)

0

In [2]:
ex_i=sorted(glob.glob("/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/*.nii.gz"))[:1000]
ex_i

['/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7014_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7017_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7018_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7029_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7030_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7031_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7032_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7033_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7034_0000.nii.gz',
 '/mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7035_0000.nii.gz',
 '/mnt/ai_neuro/augu

In [48]:
i=sitk.ReadImage("/mnt/ai_neuro/august_kun/miccai/flare24/ContourDiff/data/domain_1/contours/Case_00001_0000.nii.gz")
i_np=sitk.GetArrayFromImage(i)
np.unique(i_np)

array([-1024., -1023., -1022., ...,  3069.,  3070.,  3071.])

In [7]:
len(ex_i)

833

In [10]:
#ex_i=sorted(glob.glob("/mnt/ai_neuro/august_kun/data/domain_2/*.nii.gz"))

TypeError: glob() missing 1 required positional argument: 'pathname'

In [3]:
names=[]
shapes=[]
dtypes=[]

for file_path in ex_i:
    i = sitk.ReadImage(file_path)
    i_np = sitk.GetArrayFromImage(i)
    name = os.path.basename(file_path).split(".")[0]

    # Normalize and convert the image data to uint8 or uint16
    if i_np.dtype in [np.float32, np.float64]:
        # Normalize to 0-1 and scale to 0-255 for uint8
        i_np = (i_np - np.min(i_np)) / (np.max(i_np) - np.min(i_np))
        i_np = (i_np * 255).astype(np.uint8)  # Convert to uint8
    elif i_np.dtype in [np.int32, np.int64, np.int16]:
        # Normalize the data to fit within uint16
        i_np = np.clip(i_np, 0, 65535).astype(np.uint16)
    elif i_np.dtype == np.uint16:
        # Ensure data is within uint16 range
        i_np = np.clip(i_np, 0, 65535).astype(np.uint16)
    elif i_np.dtype == np.uint8:
        # Ensure data is within uint8 range
        i_np = np.clip(i_np, 0, 255).astype(np.uint8)
    else:
        raise TypeError(f"Unsupported image data type: {i_np.dtype}")

    
    # Ensure the output directory exists
    output_dir = "/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_1x/images/"

    # Save each slice as a separate PNG file
    for slice_index in range(i_np.shape[0]):
        slice_np = i_np[slice_index, :, :]
        zoom_factors = [512 / slice_np.shape[0], 512 / slice_np.shape[1]]
        slice_resized = zoom(slice_np, zoom_factors, order=3)
        slice_img = sitk.GetImageFromArray(slice_resized)
        slice_path = os.path.join(
            output_dir,
            f"{name}_slice{slice_index:03d}.png")
        sitk.WriteImage(slice_img, slice_path)

NiftiImageIO (0x5b10108e2290): /mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7524_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x5b10108e2290): /mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7524_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x5b10108e2290): /mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7713_0000.nii.gz has unexpected scales in sform

NiftiImageIO (0x5b10108e2290): /mnt/ai_neuro/august_kun/data/UnlabeledCase_MR/AMOS_MR_good_spacing-833/amos_7713_0000.nii.gz has unexpected scales in sform



In [5]:
pd.DataFrame({"name":names,"shape":shapes,"dtype":dtypes}).to_csv("/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/_MR_meta.csv",index=False)

In [38]:
for k in ex_m:
    i=sitk.ReadImage(k)
    i_np=sitk.GetArrayFromImage(i)
    name=os.path.basename(k).split(".")[0]

    if i_np.dtype == np.float32 or i_np.dtype == np.float64:
        i_np = (i_np - np.min(i_np)) / (np.max(i_np) - np.min(i_np))  # Normalize to 0-1
        i_np = (i_np * 255).astype(np.uint8)  # Scale to 0-255 and convert to uint8
    elif i_np.dtype == np.int32 or i_np.dtype == np.int64:
        # Convert directly to uint16 if it's safe
        i_np = np.clip(i_np, 0, 65535).astype(np.uint16)


    for k in range(i_np.shape[0]):
        slice_np = i_np[k, :, :]
        slice_img = sitk.GetImageFromArray(slice_np)
        
        # Write the slice to a PNG file
        slice_path = os.path.join("/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_2/contours/",f"{name}_slice{k:03d}.png")
        sitk.WriteImage(slice_img, slice_path)

In [11]:
p2="/mnt/ai_neuro/august_kun/miccai/flare24/ContourDiff/data/domain_2"

In [47]:
rename_images=sorted(glob.glob("/mnt/ai_neuro/august_kun/miccai/flare24/ContourDiff/data/domain_2/images/*.png"))
for i in rename_images:
    a,b=rename_images[1].split("0000")
    os.rename(i,a+b.split("_")[1])

In [18]:
import ast

In [4]:
len(glob.glob("/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_2/images/*png"))

29516

In [8]:
df.head()

,name,shape,dtype
0,amos_7014_0000,"(100, 192, 192)",uint8
1,amos_7017_0000,"(72, 290, 320)",uint16
2,amos_7018_0000,"(72, 290, 320)",uint16
3,amos_7029_0000,"(24, 192, 192)",uint8
4,amos_7030_0000,"(24, 176, 176)",uint8


In [30]:
df=pd.read_csv("/mnt/ai_neuro/august_kun/miccai/ContourDiff/data/domain_2/meta")
df.head()

,Unnamed: 0,image_name,contour_name
0,0,Case_00022_0000_slice432.png,Case_00022_0000_slice432.png
1,1,Case_00040_0000_slice079.png,Case_00040_0000_slice079.png
2,2,Case_00074_0000_slice596.png,Case_00074_0000_slice596.png
3,3,Case_00091_0000_slice005.png,Case_00091_0000_slice005.png
4,4,Case_00081_0000_slice014.png,Case_00081_0000_slice014.png


In [31]:

unique_volume_list = np.sort(df["image_name"].unique())
unique_volume_split = np.array_split(unique_volume_list,1)
unique_volume_partition = unique_volume_split[0]

In [34]:
unique_volume_partition

array(['Case_00001_0000_slice000.png', 'Case_00001_0000_slice001.png',
       'Case_00001_0000_slice002.png', ...,
       'FLARE22_Tr_0050_0000_slice092.png',
       'FLARE22_Tr_0050_0000_slice093.png',
       'FLARE22_Tr_0050_0000_slice094.png'], dtype=object)

In [32]:
len(unique_volume_partition)

29516

In [38]:
%pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 954.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [39]:
import tqdm

In [36]:
for n, volume in enumerate(unique_volume_partition):
    df_by_volume = df[df["image_name"] == volume]
    df_by_volume = df_by_volume.sort_values(by="contour_name").reset_index(drop=True)
    print(df_by_volume)

   Unnamed: 0                    image_name                  contour_name
0       22658  Case_00001_0000_slice000.png  Case_00001_0000_slice000.png
   Unnamed: 0                    image_name                  contour_name
0       13407  Case_00001_0000_slice001.png  Case_00001_0000_slice001.png
   Unnamed: 0                    image_name                  contour_name
0         788  Case_00001_0000_slice002.png  Case_00001_0000_slice002.png
   Unnamed: 0                    image_name                  contour_name
0        3926  Case_00001_0000_slice003.png  Case_00001_0000_slice003.png
   Unnamed: 0                    image_name                  contour_name
0       20007  Case_00001_0000_slice004.png  Case_00001_0000_slice004.png
   Unnamed: 0                    image_name                  contour_name
0       14204  Case_00001_0000_slice005.png  Case_00001_0000_slice005.png
   Unnamed: 0                    image_name                  contour_name
0        4217  Case_00001_0000_slice00

KeyboardInterrupt: 

In [ ]:
slice_list = df["contour_name"].to_list()
slice_split = np.array_split(slice_list, 1)
slice_partition = slice_split[0]

df_by_slice = df[df["contour_name"].isin(slice_partition)]

progress_bar_slice = tqdm(len(df_by_slice))
for i, row in df_by_slice.iterrows():
    ### Initialize the image as random noise
    batch_size = 1
    image_shape = (
        batch_size,
        args.in_channels,
        config.img_size,
        config.img_size,
    )
    image = randn_tensor(image_shape, generator=generator, device=model.device)

    ### Load contours
    contour = Image.open(os.path.join(contour_directory, row["contour_name"]))
    contour_tensor = val_transform_contour(contour)
    contour_tensor = torch.unsqueeze(contour_tensor, dim=0)
    
    scheduler.set_timesteps(config.denoise_step)

    ### Start denoising
    progress_bar_denoise = tqdm(config.denoise_step)
    for t in scheduler.timesteps:
        image = add_contours_to_noise(image, contour_tensor, config, config.device, num_copy=args.num_copy, translation=True)
        pred_noise = model(image, t).sample

        ## Extract image channel
        image = image[:, :args.in_channels, :, :]

        ## Compute image at t-1
        image = scheduler.step(pred_noise, t, image, generator=generator).prev_sample

        progress_bar_denoise.update(1)
        logs = {"denoised step": t}
        progress_bar_denoise.set_postfix(**logs)

    ### Process output and return the generated images
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()

    ### Select the generated samples with the lowest mean value
    img_list = []
    mean_list = []
    for img in image:
        img = normalize_percentile_to_255(img)

        ### Calculate mean
        mean = np.mean(img)
        mean_list.append(mean)
        img_list.append(img)
        
    index = mean_list.index(min(mean_list))
    img = img_list[index]
    img = np.squeeze(img)

    ### Save the generated samples
    img = Image.fromarray(img)
    img.save(os.path.join(save_directory, row["image_name"]))

    progress_bar_slice.update(1)
    logs = {"slice": i}
    progress_bar_slice.set_postfix(**logs)
